In [7]:
import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM 
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
import pickle

In [8]:
raw_data = pd.read_csv('2015_labeled_and_time_normalize.csv').drop(['Unnamed: 0', 'Timestamp'], axis=1)

In [9]:
headers = raw_data.columns

In [10]:
train_split_value = 250_000
val_split_values = 50_000

In [11]:
raw_data

,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1,0.0
1,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1,0.0
2,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,2,...,2,1,250.8812,1.649953,189.6789,0.000128,1,1,1,0.0
3,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,2,...,2,1,250.8812,1.649953,189.6148,0.000128,1,1,1,0.0
4,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,2,...,2,1,250.8812,1.649953,189.5027,0.000128,1,1,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449914,2.559972,519.5495,2,2,1,168.0979,8.638683,301.9226,2.459488,2,...,2,1,251.1535,0.865024,189.0220,0.000000,1,1,1,0.0
449915,2.549082,520.4131,2,2,1,168.0979,8.638683,301.9226,2.459488,2,...,2,1,251.0734,0.865024,188.9259,0.000000,1,1,1,0.0
449916,2.531467,520.6878,2,2,1,168.0979,8.638683,301.9226,2.460129,2,...,2,1,251.0734,0.865024,188.9259,0.000000,1,1,1,0.0
449917,2.521218,520.7271,2,2,1,168.0979,8.638683,301.9226,2.460129,2,...,2,1,251.0734,0.865024,188.9259,0.000000,1,1,1,0.0


In [12]:
y = raw_data['Normal/Attack']

In [13]:
data = raw_data.drop(['Normal/Attack'], axis=1)

In [14]:
#нормализация
data_headers = data.columns
normalize_data =  pd.DataFrame(preprocessing.normalize(data), columns=data_headers)

In [15]:
normalize_data

,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
0,0.001538,0.331411,0.001268,0.001268,0.000634,0.166081,0.005322,0.208307,0.001550,0.001268,...,0.000195,0.001268,0.000634,0.159013,0.001046,0.120179,8.123024e-08,0.000634,0.000634,0.000634
1,0.001551,0.331436,0.001268,0.001268,0.000634,0.166081,0.005322,0.208307,0.001550,0.001268,...,0.000195,0.001268,0.000634,0.159013,0.001046,0.120230,8.123025e-08,0.000634,0.000634,0.000634
2,0.001578,0.331392,0.001268,0.001268,0.000634,0.166072,0.005321,0.208295,0.001548,0.001268,...,0.000196,0.001268,0.000634,0.159014,0.001046,0.120223,8.122555e-08,0.000634,0.000634,0.000634
3,0.001606,0.331431,0.001268,0.001268,0.000634,0.166054,0.005320,0.208273,0.001548,0.001268,...,0.000196,0.001268,0.000634,0.158997,0.001046,0.120169,8.121678e-08,0.000634,0.000634,0.000634
4,0.001628,0.331635,0.001267,0.001267,0.000634,0.165994,0.005318,0.208198,0.001548,0.001267,...,0.000196,0.001267,0.000634,0.158940,0.001045,0.120055,8.118763e-08,0.000634,0.000634,0.000634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449914,0.001625,0.329704,0.001269,0.001269,0.000635,0.106674,0.005482,0.191599,0.001561,0.001269,...,0.000195,0.001269,0.000635,0.159381,0.000549,0.119953,0.000000e+00,0.000635,0.000635,0.000635
449915,0.001617,0.330153,0.001269,0.001269,0.000634,0.106642,0.005480,0.191542,0.001560,0.001269,...,0.000194,0.001269,0.000634,0.159283,0.000549,0.119856,0.000000e+00,0.000634,0.000634,0.000634
449916,0.001606,0.330337,0.001269,0.001269,0.000634,0.106645,0.005481,0.191547,0.001561,0.001269,...,0.000194,0.001269,0.000634,0.159287,0.000549,0.119859,0.000000e+00,0.000634,0.000634,0.000634
449917,0.001599,0.330315,0.001269,0.001269,0.000634,0.106630,0.005480,0.191520,0.001561,0.001269,...,0.000195,0.001269,0.000634,0.159265,0.000549,0.119842,0.000000e+00,0.000634,0.000634,0.000634


In [ ]:
x_train_encoder = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
val_data_encoder = val_data.reshape((val_data.shape[0], val_data.shape[1], 1))
x_test_encoder = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [121]:
LSTM_AE = Sequential()
LSTM_AE.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(x_train_encoder.shape[1], x_train_encoder.shape[2])))
LSTM_AE.add(LSTM(32, return_sequences=True, activation='relu'))
LSTM_AE.add(LSTM(16, return_sequences=True, activation='relu'))
LSTM_AE.add(LSTM(16, return_sequences=True, activation='relu'))
LSTM_AE.add(LSTM(32, return_sequences=True, activation='relu'))
LSTM_AE.add(LSTM(64, return_sequences=True, activation='relu'))
LSTM_AE.add(Dense(x_train_encoder.shape[2]))
LSTM_AE.compile(optimizer='adam', loss='mae')
LSTM_AE.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 51, 64)            16896     
_________________________________________________________________
lstm_1 (LSTM)                (None, 51, 32)            12416     
_________________________________________________________________
lstm_2 (LSTM)                (None, 51, 16)            3136      
_________________________________________________________________
lstm_3 (LSTM)                (None, 51, 16)            2112      
_________________________________________________________________
lstm_4 (LSTM)                (None, 51, 32)            6272      
_________________________________________________________________
lstm_5 (LSTM)                (None, 51, 64)            24832     
_________________________________________________________________
dense (Dense)                (None, 51, 1)             6

In [122]:
history = LSTM_AE.fit(x_train_encoder,
                      x_train_encoder,
                      validation_data=(val_data_encoder, val_data_encoder),
                      epochs=10,
                      batch_size=100,
                      shuffle=False,
                      verbose=1)

Epoch 1/10
2500/2500 [==============================] - 154s 61ms/step - loss: 0.0527 - val_loss: 0.0516
Epoch 2/10
2500/2500 [==============================] - 151s 61ms/step - loss: 0.0384 - val_loss: 0.0346
Epoch 3/10
2500/2500 [==============================] - 151s 61ms/step - loss: 0.0231 - val_loss: 0.0233
Epoch 4/10
2500/2500 [==============================] - 151s 61ms/step - loss: 0.0206 - val_loss: 0.0200
Epoch 5/10
2500/2500 [==============================] - 152s 61ms/step - loss: 0.0155 - val_loss: 0.0131
Epoch 6/10
2500/2500 [==============================] - 152s 61ms/step - loss: 0.0052 - val_loss: 0.0093
Epoch 7/10
2500/2500 [==============================] - 152s 61ms/step - loss: 0.0045 - val_loss: 0.0077
Epoch 8/10
2500/2500 [==============================] - 152s 61ms/step - loss: 0.0043 - val_loss: 0.0062
Epoch 9/10
2500/2500 [==============================] - 152s 61ms/step - loss: 0.0042 - val_loss: 0.0074
Epoch 10/10
2500/2500 [==============================] 

In [ ]:
x_train_pred = LSTM_AE.predict(x_train_encoder)

In [ ]:
train_mae_loss = np.mean(np.abs(x_train_pred - x_train_encoder), axis=1)

In [ ]:
threshold = np.max(train_mae_loss) *0.5

In [ ]:
x_test_pred = LSTM_AE.predict(x_test_encoder)

In [ ]:
test_mae_loss = np.mean(np.abs(x_test_pred - x_test_encoder), axis=1)
test_mae_loss = test_mae_loss.reshape((-1))

In [ ]:
anomalies_encoder = test_mae_loss > threshold

In [154]:
print(classification_report(y_test, anomalies_encoder))

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98     95299
         1.0       0.97      0.26      0.41      4704

    accuracy                           0.96    100003
   macro avg       0.97      0.63      0.70    100003
weighted avg       0.96      0.96      0.95    100003



In [ ]:
LSTM_AE.save('LSTM_AE.h5')